# Building a Spam Filter for SMS messages with Naive Bayes

In this short notebook, we are going to describe the [Naive Bayes algorithm](https://en.wikipedia.org/wiki/Naive_Bayes_classifier) and how it can be used to build a rudimentary spam classifier for SMS messages.

Our goal is to create a classifier that when fed an SMS message determines whether the message is spam or not. To accomplish this we'll try to compute the conditional probability that a message is spam given its contents,

$$P(spam|message)$$

We can apply [Bayes theorem](https://en.wikipedia.org/wiki/Bayes%27_theorem) to help us out here. Bayes theorem says that 

$$P(spam|message)=\frac{P(message|spam)\cdot P(spam)}{P(message)}$$

Since every message is classified as either spam or not (which we'll label 'ham'), spam and ham are complementary events, i.e. $Ham\ =\ Spam^C$. Thus, $P(ham|message)=1-P(spam|message)$.

The Naive Bayes algorithm estimates the probabilities $P(ham|message)$ and $P(spam|message)$ with some convenient (i.e. naive) conditions and then classifies the message based on which probability is greater. We'll discuss the details later. At the end of the notebook we'll evaluate our model using the `sklearn.metrics` module.

<h2>Table of Contents</h2>
<div class="alert alert-block alert-info" style="margin-top: 20px">
    <ul>
        <li>Reading and Cleaning Data</li>
        <li>The Naive Bayes Classifier</li>
        <ul><li>Additive Smoothing</li></ul>
        <li>Building The Model</li>
        <li>Prediction and Evaluation</li>
        <ul><li>Binary Classification Evaluation Metrics</li><ul>
    </ul>

</div>

<hr>


## Reading and Cleaning the Data

Now lets read in the data as a Pandas Dataframe and clean it up a bit. The dataset was put together by Tiago A. Almeida and José María Gómez Hidalgo, and it can be downloaded from the The [UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/datasets/sms+spam+collection). You can also download the dataset directly from this [link](https://dq-content.s3.amazonaws.com/433/SMSSpamCollection). The data collection process is described in more details on this [page](http://www.dt.fee.unicamp.br/~tiago/smsspamcollection/#composition), where you can also find some of the authors' papers.

In [1]:
#read in tab-seperated file and name the columns "Label" and "SMS"
import pandas as pd
messages=pd.read_csv('SMSSpamCollection.txt',sep='\t',header=None,names=['Labels','SMS'])

In [2]:
print(messages.head())

print(messages.shape)

  Labels                                                SMS
0    ham  Go until jurong point, crazy.. Available only ...
1    ham                      Ok lar... Joking wif u oni...
2   spam  Free entry in 2 a wkly comp to win FA Cup fina...
3    ham  U dun say so early hor... U c already then say...
4    ham  Nah I don't think he goes to usf, he lives aro...
(5572, 2)


So the dataframe has two columns, the plain text of the SMS message and the predetermined labeling 'Ham' for a non spam message and 'Spam' otherwise. Let's see the breakdown of messages classified as spam or not spam (i.e. ham).

In [3]:
messages['Labels'].value_counts()

ham     4825
spam     747
Name: Labels, dtype: int64

In [4]:
messages['Labels'].value_counts(normalize=True)

ham     0.865937
spam    0.134063
Name: Labels, dtype: float64

So we see that about 87% of the messages are not spam.

In order to build our spam filter we are going to split our labeled data in training and test sets by an 80-20 split. We will build our model based on the training set and then treat the test set as new messages to test our model against. We can then compare the label generated by our model to the original label.

We'll start by randomizing our dataset.

In [5]:
# The sample method on a datframe with parameter frac=1 gives us a permutation of our dataset. 
# Setting the random state allows us to reproduce our results.

messages_random=messages.sample(frac=1,random_state=1)

training_test_index = round(len(messages_random) * 0.8)

# We'll take the first 20% of entries as our training set, and the remaining as our test set.

training_set=messages_random.iloc[:training_test_index,:].reset_index(drop=True)
test_set=messages_random.iloc[training_test_index:,:].reset_index(drop=True)

In [6]:
print(training_set['Labels'].value_counts(normalize=True))
print(test_set['Labels'].value_counts(normalize=True))

ham     0.86541
spam    0.13459
Name: Labels, dtype: float64
ham     0.868043
spam    0.131957
Name: Labels, dtype: float64


Percentages looking good.

To apply the Naive Bayes  algorithm we want to count the number of unique words that appear throughout all SMS messages and count how often each word appears in each message. To make the data a bit cleaner for our model we are going to strip all punctuation from each entry in the `SMS` column of the `training_set` data frame and reduce all resulting strings to lower-case only.

In [7]:
training_set['SMS']=training_set['SMS'].str.replace('\W',' ').str.lower()
training_set.head()

,Labels,SMS
0,ham,yep by the pretty sculpture
1,ham,yes princess are you going to make me moan
2,ham,welp apparently he retired
3,ham,havent
4,ham,i forgot 2 ask ü all smth there s a card on ...


Next we'll collect all the unique word present.

In [8]:
vocabulary=[]
# Change the 'SMS' column from a string to a list of all the words present in each message
training_set['SMS']=training_set['SMS'].str.split()

In [12]:
# Create a list of all unique words in the messages
for text in training_set['SMS']:
    for word in text:
        if word not in vocabulary:
            vocabulary.append(word)

Now the list `vocabulary` contains all the unique words that appeared in the training set SMS messages. Now we'll create a disctionary whose keys are each word in `vocabulary` and whose values are lists keeping track of how often the given word occurs in each message

In [13]:
# Create a dictionary where each word is a key and the value is a list.
# the list contains the number of times each word appears in each message by index.
word_counts_per_sms={word:[0]*len(training_set['SMS']) for word in vocabulary}
for index, sms in enumerate(training_set['SMS']):
    for word in sms:
        word_counts_per_sms[word][index]+=1

In [14]:
#Convert the above dictionary to a dataframe and concatenate with the original data frame.
word_counts=pd.DataFrame(word_counts_per_sms)
training_set_clean=pd.concat([training_set,word_counts],axis=1)
training_set_clean.head()

,Labels,SMS,yep,by,the,pretty,sculpture,yes,princess,are,...,beauty,hides,secrets,n8,jewelry,related,trade,arul,bx526,wherre
0,ham,"[yep, by, the, pretty, sculpture]",1,1,1,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,ham,"[yes, princess, are, you, going, to, make, me,...",0,0,0,0,0,1,1,1,...,0,0,0,0,0,0,0,0,0,0
2,ham,"[welp, apparently, he, retired]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,ham,[havent],0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,ham,"[i, forgot, 2, ask, ü, all, smth, there, s, a,...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


The above dataframe now contains the label for each message and a column for every unique word with the number of times the given word appeared in that message. We can now compute the parameters necessary to apply the Naive Bayes classification algorithm. But what is Naive Bayes?

## The Naive Bayes Classifier

First note that 

$$P(spam|message)=\frac{P(message|spam)\cdot P(spam)}{P(message)},\ \ \ \ P(ham|message)=\frac{P(message|ham)\cdot P(ham)}{P(message)}$$

have the same denominator, so we really need only to compute $P(message|spam)\cdot P(spam)$ and $P(message|ham)\cdot P(ham)$ and compare their relative size. 

$P(spam)$ is easy to compute; we need only look at the percentage of total messages already classified as spam. In the training set, spam messages account for 86.541% of the total. But what about $P(message|spam)$? This is asking what is the probability that a given message occured knowing that it's spam a priori. Let's look at a message with one word first to illustrate.

Say we recieve the message "winner". To compute $P('winner'|spam)$ we just looka t how many times the word 'winner' occurs in all spam messages and divide by the total number of words in all spam messages. So,

$$P(spam|'winner')\propto\ P(spam)\cdot P('winner'|spam)=\frac{\#\ of\ spam\ messages}{\#\ of\ total\ messages}\cdot\frac{\#\ of\ times\ winner\ occurs\ in\ spam\ messages}{\#\ of\ words\ in\ spam\ messages}$$

So now we can compute $P(word|spam)$ for a given word and we just interpret a message as the event $w_1w_2...w_n$, where $w_i$ is the $i^{th}$ word in the message. Then by Bayes theorem we have,

$$P(spam|w_1w_2...w_n)\propto\ P(spam)\cdot P(w_1w_2...w_n|spam)$$

Now comes the 'naive' in Naive Bayes. For one, in this instantiation we've already gotten rid of all punctuation and capitalization even though spam messages may contain more grammatical errors. Furthermore, we are going to make a massive assumption that any pair of words $w_i,\ w_j$ are conditionally independent, i.e. $P(w_iw_j|spam)=P(w_i|spam)\cdot P(w_j|spam)$. This is clearly naive! Words in real messages are often in a relationship of dependence. For example, the words 'winner' and 'money' often appear in the same message.

With the above assumptions we can compute

$$P(spam|w_1w_2...w_n)\propto\ P(spam)\cdot P(w_1|spam)\cdot P(w_2|spam)\cdot...\cdot P(w_n|spam)\ =\ P(spam)\prod_{i=1}^{n} P(w_i)$$

### Additive Smoothing

There is one last caveat to consider before we code up the classifier. If we have words in a new message that only appear in one category or not at all, sometimes both $P(spam|message)$ and $P(spam^C|message)$ can evaluate to zero due to the assumed indepence conditions. To make it so none of the probabilities evaluate to zero we'll use a technique called [additive smoothing](https://en.wikipedia.org/wiki/Additive_smoothing). With the smoothing adjustment $P(w_i|Spam)$ and $P(w_i|Ham)$ take the following forms,
$$P(w_i|Spam)\ =\ \frac{N_{w_i|Spam}+\alpha}{N_{Spam}+\alpha\cdot N_{Vocabulary}}$$
$$P(w_i|Ham)\ =\ \frac{N_{w_i|Ham}+\alpha}{N_{Ham}+\alpha\cdot N_{Vocabulary}},$$

where $N_{Vocabulary}$ is the number of *unique* words appearing in all training messages. When α=1, the additive smoothing technique is most commonly known as Laplace smoothing (or add-one smoothing). However, it is also possible to use α<1, in which case the technique is called Lidstone smoothing. 

## Building the model

Now we'll compute the parameters we need to apply the Naive Bayes algorithm with Laplace smoothing.

In [15]:
#Laplace smoothing constant
alpha=1

# Computing P(ham) and P(spam)
p_ham_tr=training_set['Labels'].value_counts(normalize=True)['ham']
p_spam_tr=training_set['Labels'].value_counts(normalize=True)['spam']

In [16]:
# Seperating train messages into ham and spam
spam_messages = training_set_clean[training_set_clean['Labels'] == 'spam']
ham_messages = training_set_clean[training_set_clean['Labels'] == 'ham']

# Computing n_spam and n_ham, the total number of words in spam and ham messages respectively
num_words_per_spam=spam_messages['SMS'].apply(len)
n_spam=num_words_per_spam.sum()

num_words_per_ham=ham_messages['SMS'].apply(len)
n_ham=num_words_per_ham.sum()


In [17]:
# instantiating dictionaries that will contain {word:P(sord|spam)} and {word:P(word:ham)}
parameters_spam={unique_word:0 for unique_word in vocabulary}
parameters_ham={unique_word:0 for unique_word in vocabulary}

In [18]:
# Create series counting the number of times each word appears in spam and ham messages
spam_words=spam_messages.sum()
ham_words=ham_messages.sum()
spam_words[['the','of']]

the    157
of      79
dtype: object

In [19]:
# Use the additive smoothing formulas above to compute P(w_i|spam) and P(w_i|ham) for every unique word
# Adjust the dictionaries parameters_spam and parameters_ham accordingly

n_v=len(vocabulary)
denominator_spam=n_spam+alpha*n_v
denominator_ham=n_ham+alpha*n_v
for word in vocabulary:
    numerator_spam=spam_words[word]+alpha
    parameters_spam[word]=numerator_spam/denominator_spam
    
    numerator_ham=ham_words[word]+alpha
    parameters_ham[word]=numerator_ham/denominator_ham

In [20]:
(parameters_ham['a'],parameters_spam['a'])

(0.013395878191325745, 0.013407043050537588)

Now that we have computed

$$P(w_i|spam)\ and\ P(w_i|spam)$$

for each word in vocabulary set we can estimate the relative sizes of 

$$P(spam|new\ message)\ and\ P(ham|new\ message),$$

and the classify the new message according to which is larger!

In [21]:
#import regular expression library
import re

'''
naive_bayes will take a new message string and use the above described naive bayes algorithm
to classify the message as either spam or ham
'''

def naive_bayes(message):

    message = re.sub('\W', ' ', message)
    message = message.lower()
    message = message.split()

    

    p_spam_given_message = p_spam_tr
    p_ham_given_message = p_ham_tr
      
    for word in message:
        if word in parameters_spam:
            p_spam_given_message *= parameters_spam[word]
        if word in parameters_ham:
            p_ham_given_message *= parameters_ham[word]
        
        

    print('P(Spam|message):', p_spam_given_message)
    print('P(Ham|message):', p_ham_given_message)

    if p_ham_given_message > p_spam_given_message:
        print('Label: Ham')
    elif p_ham_given_message < p_spam_given_message:
        print('Label: Spam')
    else:
        print('Equal proabilities, have a human classify this!')

Let's see how the classifier does for the following messages which to the human eye seem to clearly be not spam and spam respectively.

In [22]:
print(test_set['SMS'][0]+'\n')
print(test_set['SMS'][1112])

Later i guess. I needa do mcat study too.

Text & meet someone sexy today. U can find a date or even flirt its up to U. Join 4 just 10p. REPLY with NAME & AGE eg Sam 25. 18 -msg recd@thirtyeight pence


In [23]:
naive_bayes(test_set['SMS'][0])
print('\n')
naive_bayes(test_set['SMS'][1112])

P(Spam|message): 3.4831070937898343e-26
P(Ham|message): 4.253245130534654e-19
Label: Ham


P(Spam|message): 2.6405759833376274e-98
P(Ham|message): 2.5002378335495614e-106
Label: Spam


Now lets change the above function a bit so it returns the classification label rather than just printing it. If the estimated values are equal, $P(spam|message)=P(ham|message)$ then the classifier informs us that we need to take a look ourselves.

In [24]:
def naive_bayes_classifier(message):

    message = re.sub('\W', ' ', message)
    message = message.lower()
    message = message.split()

    

    p_spam_given_message = n_spam
    p_ham_given_message = n_ham
      
    for word in message:
        if word in parameters_spam:
            p_spam_given_message *= parameters_spam[word]
        if word in parameters_ham:
            p_ham_given_message *= parameters_ham[word]
        
        

    

    if p_ham_given_message > p_spam_given_message:
        return 'ham'
    elif p_ham_given_message < p_spam_given_message:
        return 'spam'
    else:
        return 'requires human classification'

## Prediction and Evaluation

We can now apply our classifier to the test set as a whole. We will add a `predicted` colum to the `test_set` dataframe. We'll then also add a Boolean column `correct` to keep track of whether or not the classifier chose the correct label.

In [25]:
test_set['predicted']=test_set['SMS'].apply(naive_bayes_classifier)

In [26]:
test_set.head()

,Labels,SMS,predicted
0,ham,Later i guess. I needa do mcat study too.,ham
1,ham,But i haf enuff space got like 4 mb...,ham
2,spam,Had your mobile 10 mths? Update to latest Oran...,spam
3,ham,All sounds good. Fingers . Makes it difficult ...,ham
4,ham,"All done, all handed in. Don't know if mega sh...",ham


In [27]:
test_set['predicted'].value_counts()

ham                              968
spam                             145
requires human classification      1
Name: predicted, dtype: int64

So we see that the classifier wasn't able to classify one of the messages so we'll have to do it ourselves!

In [28]:
test_set[test_set['predicted']=='requires human classification']['SMS']

293    A Boy loved a gal. He propsd bt she didnt mind...
Name: SMS, dtype: object

Difficult to say! Given the romantic nature and mispelled words I think I would personally see it as spam.

In [29]:
test_set.loc[293,'predicted']='spam'
test_set['predicted'].value_counts()

ham     968
spam    146
Name: predicted, dtype: int64

So now every message is classified. We'll also add a Boolean `correct` coulmn to see if our prediction matches with the label from the original dataset.

In [30]:
test_set['correct']=(test_set['Labels']==test_set['predicted'])

In [31]:
test_set.head()

,Labels,SMS,predicted,correct
0,ham,Later i guess. I needa do mcat study too.,ham,True
1,ham,But i haf enuff space got like 4 mb...,ham,True
2,spam,Had your mobile 10 mths? Update to latest Oran...,spam,True
3,ham,All sounds good. Fingers . Makes it difficult ...,ham,True
4,ham,"All done, all handed in. Don't know if mega sh...",ham,True


Now let's look at some accuracy metrics to evaluate our model!

### Binary Classification Evaluation Metrics

To wrap things up we'll discuss some metrics for evaluating binary classifiers.

A stright forward metric is prediction accuracy. We simply want to know how many predictions did we get correct.

$$Prediction\ Accuracy\ =\ \frac{Number\ of\ Correct\ Predictions}{Number\ of\ Total\ Predictions}$$

We also want to look at precision and recall, also know as the true positive and true negative rates respectively.

In [32]:
from sklearn.metrics import classification_report, accuracy_score

In [33]:
accuracy_score(test_set['Labels'],test_set['predicted'])

0.9865350089766607

So our model classified about 98% of the messages correctly. Accuracy alone doesn't discriminate between the different types of binary classifications. Since we're trying to find spam messages, we'll take a spam classification as 'positive' and ham as 'negative. Then there are 4 kinds of binary classifications detailed in the table below,

|Prediction|Observation|            |
|----------|-----------|------------|
|          |Spam      |Ham  |
|Spam     |True Positive (TP)|False Positive (FP)|
|Ham       |False Negative (FM)|True Negative (TN)|

There are some binary classification measures that are more subtle than accuracy alone. For example, we could look at **sensitivity**, or the true positive rate,

$$Sensitivity\ =\ \frac{True\ Positives}{True\ Positives\ +\ False\ Negatives}.$$

Of all the messages that should have been classified as spam, how many were correctly labeled as such. Sensitivity is also known as **recall**. Then we also  have **specificity**, or the true negative rate,

$$Specificity\ =\ \frac{True\ Negatives}{True\ Negatives\ +\ False\ Positives}$$

Of all the messages that should have gotten through our filter (i.e. ham), what percentage did make it to pur inbox. We also have **precision**,

$$Precision\ =\ \frac{True\ Positives}{True\ Positives\ +\ False\ Positives},$$

which measures how many of the spam predictions were relevant. Different models may purposely try to maximize sensitivity relative to recall or vice-versa. In our case I would prefer to maximise specificity as I personally think it more important that all non-spam messages make it into my inbox at the expense of the occasional spam message getting through the filter. These measures can be found easily using the `sklearn.metrics.classification_report()` method.

In [34]:
print(classification_report(test_set['Labels'],test_set['predicted']))

              precision    recall  f1-score   support

         ham       0.99      0.99      0.99       967
        spam       0.95      0.95      0.95       147

    accuracy                           0.99      1114
   macro avg       0.97      0.97      0.97      1114
weighted avg       0.99      0.99      0.99      1114



The `f1-score` is the harmonic mean of precision and sensitivity.

How might we make our model better? We could make our model more complex by making it sensitive to case and punctuation. Looking at the messages the were classified incorrectly, this seems like a good bet!

In [38]:
test_set[test_set['correct']==False]

,Labels,SMS,predicted,correct
114,spam,Not heard from U4 a while. Call me now am here...,ham,False
135,spam,More people are dogging in your area now. Call...,ham,False
152,ham,Unlimited texts. Limited minutes.,spam,False
159,ham,26th OF JULY,spam,False
284,ham,Nokia phone is lovly..,spam,False
293,ham,A Boy loved a gal. He propsd bt she didnt mind...,spam,False
302,ham,No calls..messages..missed calls,spam,False
319,ham,We have sent JD for Customer Service cum Accou...,spam,False
331,ham,Just taste fish curry :-P,spam,False
504,spam,Oh my god! I've found your number again! I'm s...,ham,False
